### Cleaning Fighters Data

In [17]:
import pandas as pd
import numpy as np

In [18]:
fighters_df = pd.read_csv('data/fighters.csv')

In [ ]:
fighters_df.head()

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
0,Tom,Aaron,NaN,--,155 lbs.,--,NaN,5,3,0,NaN
1,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4,6,0,NaN
2,Nariman,Abbasov,Bayraktar,"5' 8""",155 lbs.,"66.0""",Orthodox,28,4,0,NaN
3,Darion,Abbey,NaN,"6' 2""",265 lbs.,"80.0""",Orthodox,9,5,0,NaN
4,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10,15,0,NaN


In [34]:
def check_types():
    print(fighters_df.dtypes)

In [21]:
# Ht. column
def format_height(height=""):
    if height == '--':
        return np.nan
    height = height.rstrip('"')
    height = height.replace("' ", ".")
    return height


fighters_df['Ht.'] = fighters_df['Ht.'].apply(format_height)
fighters_df['Ht.'] = fighters_df['Ht.'].astype('float32')

In [27]:
# Wt. column
def format_weight(height=""):
    if height == '--':
        return np.nan
    height = height.rstrip(' lbs.')
    return height


fighters_df['Wt.'] = fighters_df['Wt.'].apply(format_weight)
fighters_df['Wt.'] = fighters_df['Wt.'].astype('float32')

In [ ]:
# Reach column
def format_reach(height=""):
    if height == '--':
        return np.nan
    height = height.rstrip('"')
    return height


fighters_df['Reach'] = fighters_df['Reach'].apply(format_reach)
fighters_df['Reach'] = fighters_df['Reach'].astype('float32')

In [42]:
# Making stance column categorical
fighters_df['Stance'] = fighters_df['Stance'].astype('category')

In [37]:
# W - L - D can not be so big.
# It is better to make them only 32-bits
for col in ['W','L','D'] : 
    fighters_df[col] = fighters_df[col].astype('int32')

In [41]:
fighters_df.head()

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
0,Tom,Aaron,NaN,NaN,155.0,NaN,NaN,5,3,0,NaN
1,Danny,Abbadi,The Assassin,5.11,155.0,NaN,Orthodox,4,6,0,NaN
2,Nariman,Abbasov,Bayraktar,5.80,155.0,66.0,Orthodox,28,4,0,NaN
3,Darion,Abbey,NaN,6.20,265.0,80.0,Orthodox,9,5,0,NaN
4,David,Abbott,Tank,6.00,265.0,NaN,Switch,10,15,0,NaN


I think this is enough for cleaning fighters data

In [52]:
fighters_df.to_csv('data/Fighters.csv')